In [1]:
import os
from os import chdir

In [2]:
%pwd

'd:\\End-to-End-Text-Summerizer_NLP_Project\\research'

In [3]:
chdir("../")

In [4]:
%pwd

'd:\\End-to-End-Text-Summerizer_NLP_Project'

In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path
 

In [20]:
from src.EndtoEndTextSummerizer.constants import *
from src.EndtoEndTextSummerizer.utils.common import read_yaml, create_directories

In [21]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config  = self.config.data_transformation

        data_transformation_config = DataTransformationConfig(
            root_dir= config.root_dir,
            data_path=config.data_path,
            tokenizer_name= config.tokenizer_name
        )
        return data_transformation_config

In [22]:
import os
from src.EndtoEndTextSummerizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [23]:
class DataTransformation:
    def __init__(self,config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features,batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [24]:
load_from_disk('file://artifacts/data_ingestion/samsum_dataset/')


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [25]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [26]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transforamtion = DataTransformation(config = data_transformation_config)
    data_transforamtion.convert()
except Exception as e:
    raise e

[2024-01-07 17:34:49,134: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-07 17:34:49,153: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-07 17:34:49,156: INFO: common: created directory at: artifacts]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

c:\Users\DELL\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]